In [ ]:
!pip install osmnx geopandas pandas openpyxl shapely

In [ ]:
import pandas as pd
import numpy as np
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, box
import warnings
import time
from typing import Dict, List, Tuple
import sys

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

BATCH_SIZE = 5
SEARCH_RADIUS = 5000  # meters = 5KM
OUTPUT_FILE = "Air_Quality_with_OSM_Features_5KM.xlsx"
INPUT_FILE = r"data/processed/india_aq_transformed_last30days.csv"

# ============================================================================
# FEATURE EXTRACTION FUNCTIONS
# ============================================================================

def extract_roads(lat: float, lon: float, radius: int) -> Dict:
    """Extract road features from OSM"""
    try:
        roads = ox.features_from_point(
            (lat, lon),
            tags={'highway': True},
            dist=radius
        )
        
        if len(roads) > 0:
            road_names = [roads.index.get_level_values(0).unique()[i] 
                         for i in range(min(5, len(roads.index.get_level_values(0).unique())))]
            return {
                'count': len(roads),
                'examples': ', '.join(str(r)[:40] for r in road_names),
                'status': 'success'
            }
        return {'count': 0, 'examples': '', 'status': 'success'}
    except Exception as e:
        return {'count': 0, 'examples': f"Error: {str(e)[:30]}", 'status': 'failed'}

def extract_industrial_zones(lat: float, lon: float, radius: int) -> Dict:
    """Extract industrial facilities from OSM"""
    try:
        tags = {'industrial': True, 'landuse': 'industrial'}
        industrial = ox.features_from_point((lat, lon), tags=tags, dist=radius)
        
        if len(industrial) > 0:
            ind_names = industrial.index.get_level_values(0).unique()[:5]
            return {
                'count': len(industrial),
                'examples': ', '.join(str(i)[:35] for i in ind_names),
                'status': 'success'
            }
        return {'count': 0, 'examples': '', 'status': 'success'}
    except Exception as e:
        return {'count': 0, 'examples': f"Error: {str(e)[:30]}", 'status': 'failed'}

def extract_dump_sites(lat: float, lon: float, radius: int) -> Dict:
    """Extract waste management and dump sites from OSM"""
    try:
        tags = {'waste': True}
        dumps = ox.features_from_point((lat, lon), tags=tags, dist=radius)
        
        if len(dumps) > 0:
            dump_names = dumps.index.get_level_values(0).unique()[:5]
            return {
                'count': len(dumps),
                'examples': ', '.join(str(d)[:35] for d in dump_names),
                'status': 'success'
            }
        return {'count': 0, 'examples': '', 'status': 'success'}
    except Exception as e:
        return {'count': 0, 'examples': f"Error: {str(e)[:30]}", 'status': 'failed'}

def extract_agricultural_fields(lat: float, lon: float, radius: int) -> Dict:
    """Extract agricultural areas from OSM"""
    try:
        tags = {'landuse': ['farmland', 'farm', 'agricultural', 'grass', 'meadow']}
        agriculture = ox.features_from_point((lat, lon), tags=tags, dist=radius)
        
        if len(agriculture) > 0:
            agr_names = agriculture.index.get_level_values(0).unique()[:5]
            return {
                'count': len(agriculture),
                'examples': ', '.join(str(a)[:35] for a in agr_names),
                'status': 'success'
            }
        return {'count': 0, 'examples': '', 'status': 'success'}
    except Exception as e:
        return {'count': 0, 'examples': f"Error: {str(e)[:30]}", 'status': 'failed'}

def extract_all_features(lat: float, lon: float, radius: int) -> Dict:
    """Extract all features for a location"""
    features = {
        'roads': extract_roads(lat, lon, radius),
        'industrial': extract_industrial_zones(lat, lon, radius),
        'dumps': extract_dump_sites(lat, lon, radius),
        'agriculture': extract_agricultural_fields(lat, lon, radius)
    }
    return features

# ============================================================================
# MAIN PROCESSING
# ============================================================================

def main():
    print(f"\n{'='*80}")
    print("OSM FEATURE EXTRACTION FOR AIR QUALITY ANALYSIS")
    print(f"{'='*80}\n")
    
    # Load dataset
    print(f"Loading dataset from {INPUT_FILE}...")
    df = pd.read_csv(INPUT_FILE)
    print(f"✓ Loaded {len(df)} records\n")
    
    # Get unique locations
    unique_locations = df[['latitude', 'longitude','location_name', 'state',
                           'district']].drop_duplicates(
                           subset=['latitude', 'longitude']).reset_index(drop=True)
    
    print(f"Unique locations to process: {len(unique_locations)}")
    print(f"Processing parameters:")
    print(f"  - Batch size: {BATCH_SIZE}")
    print(f"  - Search radius: {SEARCH_RADIUS}m")
    print(f"\n{'='*80}\n")
    
    # Initialize results
    features_data = []
    total_batches = (len(unique_locations) - 1) // BATCH_SIZE + 1
    
    # Process in batches
    for batch_num in range(0, len(unique_locations), BATCH_SIZE):
        batch = unique_locations.iloc[batch_num:batch_num + BATCH_SIZE]
        batch_idx = (batch_num // BATCH_SIZE) + 1
        
        print(f"Batch {batch_idx}/{total_batches}:")
        print(f"Processing locations {batch_num + 1} to {min(batch_num + BATCH_SIZE, len(unique_locations))}\n")
        
        for idx, row in batch.iterrows():
            lat = row['latitude']
            lon = row['longitude']
            location_name = row['location_name']
            city = row['district']
            state = row['state']
            
            try:
                # Extract features
                print(f"  Querying {city}, {state} ({lat:.4f}, {lon:.4f})...", end='', flush=True)
                features = extract_all_features(lat, lon, SEARCH_RADIUS)
                
                # Compile record
                record = {
                    'location_name': location_name,
                    'district': city,
                    'state': state,
                    'latitude': lat,
                    'longitude': lon,
                    'Roads_count': features['roads']['count'],
                    'Roads_examples': features['roads']['examples'],
                    'Industrial_zones_count': features['industrial']['count'],
                    'Industrial_examples': features['industrial']['examples'],
                    'Dump_sites_count': features['dumps']['count'],
                    'Dump_examples': features['dumps']['examples'],
                    'Agricultural_fields_count': features['agriculture']['count'],
                    'Agricultural_examples': features['agriculture']['examples'],
                    'Query_status': 'Success'
                }
                
                features_data.append(record)
                
                print(f" ✓ Roads:{features['roads']['count']} | "
                      f"Industrial:{features['industrial']['count']} | "
                      f"Dumps:{features['dumps']['count']} | "
                      f"Agriculture:{features['agriculture']['count']}")
                
            except Exception as e:
                print(f" ✗ Error: {str(e)[:50]}")
                record = {
                    'location_name': location_name,
                    'district': city,
                    'state': state,
                    'latitude': lat,
                    'longitude': lon,
                    'Roads_count': 0,
                    'Roads_examples': '',
                    'Industrial_zones_count': 0,
                    'Industrial_examples': '',
                    'Dump_sites_count': 0,
                    'Dump_examples': '',
                    'Agricultural_fields_count': 0,
                    'Agricultural_examples': '',
                    'Query_status': f'Failed: {str(e)[:40]}'
                }
                features_data.append(record)
            
            # Small delay between queries to avoid rate limiting
            time.sleep(0.5)
        
        print(f"\nBatch completed. Waiting before next batch...\n")
        time.sleep(2)
    
    # Create features dataframe
    features_df = pd.DataFrame(features_data)
    
    # Print summary statistics
    print(f"\n{'='*80}")
    print("EXTRACTION SUMMARY")
    print(f"{'='*80}\n")
    
    print(f"Total locations processed: {len(features_df)}")
    successful = (features_df['Query_status'] == 'Success').sum()
    print(f"Successful queries: {successful}/{len(features_df)}")
    
    print(f"\n{'Feature Statistics':^80}")
    print(f"{'-'*80}")
    print(f"{'Feature':<20} {'Total':<12} {'Mean':<12} {'Max':<12}")
    print(f"{'-'*80}")
    print(f"{'Roads':<20} {features_df['Roads_count'].sum():<12.0f} "
          f"{features_df['Roads_count'].mean():<12.1f} {features_df['Roads_count'].max():<12.0f}")
    print(f"{'Industrial':<20} {features_df['Industrial_zones_count'].sum():<12.0f} "
          f"{features_df['Industrial_zones_count'].mean():<12.1f} "
          f"{features_df['Industrial_zones_count'].max():<12.0f}")
    print(f"{'Dump Sites':<20} {features_df['Dump_sites_count'].sum():<12.0f} "
          f"{features_df['Dump_sites_count'].mean():<12.1f} {features_df['Dump_sites_count'].max():<12.0f}")
    print(f"{'Agriculture':<20} {features_df['Agricultural_fields_count'].sum():<12.0f} "
          f"{features_df['Agricultural_fields_count'].mean():<12.1f} "
          f"{features_df['Agricultural_fields_count'].max():<12.0f}")
    print(f"{'-'*80}\n")
    
    # Merge with original data
    print("Merging extracted features with original air quality data...")
    
    # Merge on latitude and longitude
    merged_df = df.merge(
        features_df[['latitude', 'longitude', 'Roads_count', 'Industrial_zones_count',
                     'Dump_sites_count', 'Agricultural_fields_count', 'Query_status']],
        on=['latitude', 'longitude'],
        how='left'
    )
    
    # Fill NaN values for unmatched records (should be minimal)
    merged_df.fillna({
    'Roads_count': 0,
    'Industrial_zones_count': 0,
    'Dump_sites_count': 0,
    'Agricultural_fields_count': 0,
    'Query_status': 'Not processed'
     }, inplace=True)
    
    # Create additional derived features
    merged_df['Urban_density_score'] = (merged_df['Roads_count'] / merged_df['Roads_count'].max()).round(2)
    merged_df['Industrial_presence'] = (merged_df['Industrial_zones_count'] > 0).astype(int)
    merged_df['Pollution_source_risk'] = (
        (merged_df['Industrial_zones_count'] * 0.4 + 
         merged_df['Dump_sites_count'] * 0.3 + 
         merged_df['Roads_count'] * 0.3) / 100
    ).round(2)
    merged_df['Green_area_ratio'] = (
        merged_df['Agricultural_fields_count'] / 
        (merged_df['Roads_count'] + merged_df['Agricultural_fields_count'])
    ).round(2)
    
    # Save to Excel
    print(f"\nSaving merged data to {OUTPUT_FILE}...")
    with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
        merged_df.to_excel(writer, sheet_name='All Data', index=False)
        features_df.to_excel(writer, sheet_name='OSM Features', index=False)
        
        # Summary statistics sheet
        summary_stats = pd.DataFrame({
            'Metric': ['Total Records', 'Unique Locations', 'Successful Queries',
                      'Mean Roads', 'Mean Industrial', 'Mean Dumps', 'Mean Agriculture',
                      'Max Roads', 'Max Industrial', 'Max Dumps', 'Max Agriculture'],
            'Value': [len(merged_df), len(features_df), successful,
                     f"{features_df['Roads_count'].mean():.1f}",
                     f"{features_df['Industrial_zones_count'].mean():.1f}",
                     f"{features_df['Dump_sites_count'].mean():.1f}",
                     f"{features_df['Agricultural_fields_count'].mean():.1f}",
                     features_df['Roads_count'].max(),
                     features_df['Industrial_zones_count'].max(),
                     features_df['Dump_sites_count'].max(),
                     features_df['Agricultural_fields_count'].max()]
        })
        summary_stats.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"✓ Successfully saved to {OUTPUT_FILE}\n")
    
    # Display sample results
    print(f"{'Sample Results (First 15 locations):':^80}")
    print(merged_df[['district', 'state', 'Roads_count',
                     'Industrial_zones_count', 'Dump_sites_count', 'Agricultural_fields_count',
                     'Pollution_source_risk', 'Green_area_ratio']].head(15))
    
    print(f"\n{'='*80}")
    print("✓ Processing Complete!")
    print(f"{'='*80}\n")
    
    return merged_df, features_df

if __name__ == "__main__":
    merged_df, features_df = main()

In [ ]:
import pandas as pd
df = pd.read_excel("Air_Quality_with_OSM_Features_5KM.xlsx")
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_excel("Air_Quality_with_OSM_Features_5KM.xlsx")

# 1. Remove records with invalid coordinates
df = df[(df["latitude"].between(-90, 90)) & (df["longitude"].between(-180, 180))]

# 2. Remove records with missing timestamps
df = df.dropna(subset=["datetime_utc"])

# 3. Remove records where ALL pollutant values are missing
pollutant_cols = ["pm25", "pm10", "no2", "so2", "o3", "co"]  # adjust to your dataset
df = df.dropna(how="all", subset=pollutant_cols)

# 4. Remove outliers (values outside 3 standard deviations)
for col in pollutant_cols:
    if col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        df = df[(df[col] >= mean - 3*std) & (df[col] <= mean + 3*std) | df[col].isna()]

# 5. Only filter if values are unrealistic (negative or extremely high)
for col in pollutant_cols:
    if col in df.columns:
        df = df[(df[col] >= 0) & (df[col] <= 500) | df[col].isna()]  # AQI realistic range

In [ ]:
df.isnull().sum()

In [ ]:
pollutant_cols = ["pm25", "pm10", "no2", "co", "so2", "o3"]
weather_cols=['temperature','humidity','wind_speed','wind_direction']

for col in pollutant_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col]=df[col].fillna(median_val)
for col in weather_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col]=df[col].fillna(median_val)        

print(df[pollutant_cols].isnull().sum())
print(df[weather_cols].isnull().sum())

In [ ]:
print("Standarsdizing timestamps")

# Convert datetime_utc to datetime object
print("\nConverting datetime strings to datetime objects...")
df['datetime_utc'] = pd.to_datetime(df['datetime_utc'], utc=True)

# Convert to IST (Indian Standard Time) for local analysis
df['datetime_ist'] = df['datetime_utc'].dt.tz_convert('Asia/Kolkata')

# Extract date components
print("Extracting date components...")
df['date'] = df['datetime_ist'].dt.date
df['year'] = df['datetime_ist'].dt.year
df['month'] = df['datetime_ist'].dt.month
df['day'] = df['datetime_ist'].dt.day
df['hour'] = df['datetime_ist'].dt.hour
df['day_of_week'] = df['datetime_ist'].dt.dayofweek  # 0=Monday, 6=Sunday
df['day_name'] = df['datetime_ist'].dt.day_name()
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Round coordinates to standard precision
print("Standardizing GPS coordinates...")

df['latitude'] = df['latitude'].round(6)
df['longitude'] = df['longitude'].round(6)

print("Sample of standardized data:")
print("-"*50)
print(df[['datetime_ist', 'date', 'hour', 'day_of_week', 'is_weekend']].head())



In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

df = pd.read_excel("Air_Quality_with_OSM_Features_5KM.xlsx")

# Define columns
pollutant_cols = ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3']
weather_cols = ['temperature', 'humidity', 'wind_speed', 'wind_direction']
cols_to_normalize = [col for col in pollutant_cols + weather_cols if col in df.columns]

print("Columns to normalize:", cols_to_normalize)

# --- Min-Max Normalization (0-1 range) ---
print("\nApplying Min-Max normalization (0-1 range)...")
minmax_scaler = MinMaxScaler()
for col in cols_to_normalize:
    df[col + "_normalized"] = minmax_scaler.fit_transform(df[[col]])
    print(f"  ✓ {col}_normalized created")

# --- Standard Scaling (Z-score) ---
print("\nApplying Standard scaling (z-score)...")
standard_scaler = StandardScaler()
for col in cols_to_normalize:
    df[col + "_scaled"] = standard_scaler.fit_transform(df[[col]])
    print(f"  ✓ {col}_scaled created")

print("\n✅ Normalization complete!")

# --- Sample Output for one feature (pm25) ---
print("\n--------------------------------------------------")
print("Sample Normalized Values:")
print("--------------------------------------------------")
print(df[['pm25', 'pm25_normalized', 'pm25_scaled']].describe())

In [ ]:
df.head()


In [ ]:
import sys
print(sys.executable)


In [ ]:
from pyrosm import OSM
print("pyrosm imported successfully")


In [1]:
import os

os.path.exists(r"data\roads.geojson"), os.path.exists(r"data\industrial.geojson")


(True, True)

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import box
from shapely.strtree import STRtree
import numpy as np
from tqdm import tqdm

# ---------------------------
# PATHS
# ---------------------------
INPUT_EXCEL = "Air_Quality_with_OSM_Features_5KM.xlsx"
OUTPUT_EXCEL = "dataset_with_spatial_proximity_features.xlsx"

roads_fp = r"data/roads.osm.pbf"
industrial_fp = r"data/industrial.osm.pbf"
dump_fp = r"data/dump_sites.osm.pbf"
agri_fp = r"data/agriculture.osm.pbf"

# ---------------------------
# 1. LOAD INPUT DATA
# ---------------------------
print("📥 Loading input Excel...")
df = pd.read_excel(INPUT_EXCEL)
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"
).to_crs(epsg=3857)

points = gdf.geometry.values

# ---------------------------
# 2. LOAD OSM FEATURES (SMALL PBFs)
# ---------------------------
print("🗺️ Loading OSM features from small PBFs...")

roads = gpd.read_file(roads_fp).to_crs(3857)
industrial = gpd.read_file(industrial_fp).to_crs(3857)
dump_sites = gpd.read_file(dump_fp).to_crs(3857)
agriculture = gpd.read_file(agri_fp).to_crs(3857)

# ---------------------------
# 3. FILTER FEATURES TO POINTS BBOX
# ---------------------------
minx, miny, maxx, maxy = gdf.total_bounds
bbox_poly = box(minx, miny, maxx, maxy)

roads = roads[roads.intersects(bbox_poly)]
industrial = industrial[industrial.intersects(bbox_poly)]
dump_sites = dump_sites[dump_sites.intersects(bbox_poly)]
agriculture = agriculture[agriculture.intersects(bbox_poly)]

# ---------------------------
# 4. BUILD STRtree INDEXES
# ---------------------------
road_tree = STRtree(roads.geometry.values)
ind_tree = STRtree(industrial.geometry.values)
dump_tree = STRtree(dump_sites.geometry.values)
agri_tree = STRtree(agriculture.geometry.values)

# ---------------------------
# 5. VECTORIZE DISTANCE FUNCTION
# ---------------------------
def min_distance_vectorized(tree, points):
    result = np.zeros(len(points))
    for i, p in enumerate(points):
        candidates = tree.query(p)
        if len(candidates) == 0:
            result[i] = np.nan
        else:
            dists = np.array([p.distance(g) for g in candidates])
            result[i] = dists.min()
    return result

# ---------------------------
# 6. BATCH PROCESSING (SAFE FOR MEMORY)
# ---------------------------
batch_size = 5000
n = len(points)

dist_road = np.zeros(n)
dist_ind = np.zeros(n)
dist_dump = np.zeros(n)
dist_agri = np.zeros(n)

print("⏳ Computing distances in batches...")
for start in tqdm(range(0, n, batch_size), desc="Batches"):
    end = min(start + batch_size, n)
    batch_points = points[start:end]

    dist_road[start:end] = min_distance_vectorized(road_tree, batch_points)
    dist_ind[start:end] = min_distance_vectorized(ind_tree, batch_points)
    dist_dump[start:end] = min_distance_vectorized(dump_tree, batch_points)
    dist_agri[start:end] = min_distance_vectorized(agri_tree, batch_points)

# ---------------------------
# 7. SAVE OUTPUT
# ---------------------------
gdf["dist_nearest_road_m"] = dist_road
gdf["dist_nearest_industry_m"] = dist_ind
gdf["dist_nearest_dump_m"] = dist_dump
gdf["dist_nearest_agriculture_m"] = dist_agri

print("💾 Saving final dataset...")
gdf.drop(columns="geometry").to_excel(OUTPUT_EXCEL, index=False)

print("✅ DONE — Spatial proximity features computed successfully!")


📥 Loading input Excel...
🗺️ Loading OSM features from small PBFs...


C:\Riya\Batch-6\miniconda3\envs\geo\lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'roads.osm.pbf': 'points' (default), 'lines', 'multilinestrings', 'multipolygons', 'other_relations'. Specify layer parameter to avoid this warning.
  result = read_func(
C:\Riya\Batch-6\miniconda3\envs\geo\lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'industrial.osm.pbf': 'points' (default), 'lines', 'multilinestrings', 'multipolygons', 'other_relations'. Specify layer parameter to avoid this warning.
  result = read_func(
C:\Riya\Batch-6\miniconda3\envs\geo\lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'dump_sites.osm.pbf': 'points' (default), 'lines', 'multilinestrings', 'multipolygons', 'other_relations'. Specify layer parameter to avoid this warning.
  result = read_func(
C:\Riya\Batch-6\miniconda3\envs\geo\lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one laye

⏳ Computing distances in batches...


Batches: 100%|█████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.67it/s]


💾 Saving final dataset...
✅ DONE — Spatial proximity features computed successfully!


In [3]:
# ============================================================
# SINGLE-CELL: TEMPORAL FEATURE ENGINEERING (EXCEL)
# ============================================================

import pandas as pd

# -----------------------------
# 1. Load Excel file
# -----------------------------
df = pd.read_excel("dataset_with_spatial_proximity_features.xlsx")

# -----------------------------
# 2. Parse datetime column
# -----------------------------
df["datetime_local"] = pd.to_datetime(df["datetime_local"])

# -----------------------------
# 3. Hour-based features
# -----------------------------
df["hour"] = df["datetime_local"].dt.hour
df["is_peak_hour"] = df["hour"].isin([7, 8, 9, 18, 19, 20]).astype(int)

# -----------------------------
# 4. Day-based features
# -----------------------------
df["day_of_week"] = df["datetime_local"].dt.dayofweek   # 0=Mon, 6=Sun
df["is_weekend"] = (df["day_of_week"] >= 5).astype(int)

# -----------------------------
# 5. Month & season features (India)
# -----------------------------
df["month"] = df["datetime_local"].dt.month

def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Summer"
    elif month in [6, 7, 8, 9]:
        return "Monsoon"
    else:
        return "Post_Monsoon"

df["season"] = df["month"].apply(get_season)

# Numeric encoding (ML-ready)
season_map = {
    "Winter": 0,
    "Summer": 1,
    "Monsoon": 2,
    "Post_Monsoon": 3
}
df["season_code"] = df["season"].map(season_map)

# -----------------------------
# 6. Remove timezone info
# -----------------------------
df["datetime_local"] = df["datetime_local"].dt.tz_localize(None)

# -----------------------------
# 7. Save back to Excel
# -----------------------------
df.to_excel(
    "dataset_with_spatial_and_temporal_features.xlsx",
    index=False
)

print("✅ Temporal features added and saved to Excel!")


✅ Temporal features added and saved to Excel!


In [4]:
# ============================================================
# MEMORY-EFFICIENT NORMALIZATION OF POLLUTANT & WEATHER FEATURES
# ============================================================

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

# -----------------------------
# 1. Load dataset
# -----------------------------
INPUT_FILE = "dataset_with_spatial_and_temporal_features.xlsx"
OUTPUT_FILE = "Final_dataset.xlsx"

df = pd.read_excel(INPUT_FILE)
print(f"📥 Loaded dataset ({df.shape[0]} rows, {df.shape[1]} columns)")

# -----------------------------
# 2. Define columns
# -----------------------------
pollutant_cols = ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3']
weather_cols = ['temperature', 'humidity', 'wind_speed', 'wind_direction']

cols_to_normalize = [col for col in pollutant_cols + weather_cols if col in df.columns]
print("Columns to normalize:", cols_to_normalize)

# Convert to float32 to save memory
df[cols_to_normalize] = df[cols_to_normalize].astype(np.float32)

# -----------------------------
# 3. Handle missing values (median)
# -----------------------------
df[cols_to_normalize] = df[cols_to_normalize].fillna(df[cols_to_normalize].median())
print("✅ Missing values filled with median")

# -----------------------------
# 4. Min-Max Normalization (0–1)
# -----------------------------
minmax_scaler = MinMaxScaler()
df[[f"{col}_normalized" for col in cols_to_normalize]] = minmax_scaler.fit_transform(df[cols_to_normalize])
print("✅ Min-Max normalization done")

# -----------------------------
# 5. Standard Scaling (Z-score)
# -----------------------------
standard_scaler = StandardScaler()
df[[f"{col}_scaled" for col in cols_to_normalize]] = standard_scaler.fit_transform(df[cols_to_normalize])
print("✅ Standard scaling done")

# -----------------------------
# 6. Save final dataset
# -----------------------------
df.to_excel(OUTPUT_FILE, index=False)
print(f"\n💾 Dataset saved: {OUTPUT_FILE}")

# -----------------------------
# 7. Quick check
# -----------------------------
print("\nSample (PM2.5):")
print(df[['pm25', 'pm25_normalized', 'pm25_scaled']].describe())


📥 Loaded dataset (77994 rows, 38 columns)
Columns to normalize: ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3', 'temperature', 'humidity', 'wind_speed', 'wind_direction']
✅ Missing values filled with median
✅ Min-Max normalization done
✅ Standard scaling done

💾 Dataset saved: Final_dataset.xlsx

Sample (PM2.5):
                pm25  pm25_normalized   pm25_scaled
count   77994.000000     77994.000000  7.799400e+04
mean      180.638641         0.011834 -1.271664e-09
std      6087.090820         0.007076  1.000006e+00
min     -9999.000000         0.000000 -1.672343e+00
25%        29.570000         0.011659 -2.481803e-02
50%        46.279999         0.011678 -2.207286e-02
75%        73.757502         0.011710 -1.755877e-02
max    850191.250000         1.000000  1.396424e+02


In [1]:
# ============================================================
# SAFE MERGE: SPATIAL-TEMPORAL + NORMALIZED DATA
# ============================================================

import pandas as pd

# -----------------------------
# 1. Load datasets
# -----------------------------
spatial_temporal = pd.read_excel("dataset_with_spatial_and_temporal_features.xlsx")
normalized = pd.read_excel("Final_dataset.xlsx")

# -----------------------------
# 2. Remove timezone from datetime (if any)
# -----------------------------
if isinstance(spatial_temporal["datetime_local"].dtype, pd.DatetimeTZDtype):
    spatial_temporal["datetime_local"] = spatial_temporal["datetime_local"].dt.tz_localize(None)

if isinstance(normalized["datetime_local"].dtype, pd.DatetimeTZDtype):
    normalized["datetime_local"] = normalized["datetime_local"].dt.tz_localize(None)


# -----------------------------
# 3. Define merge columns
# -----------------------------
merge_cols = ["location_id", "datetime_local"]

# -----------------------------
# 4. Merge datasets
# -----------------------------
final_df = pd.merge(
    spatial_temporal,
    normalized,
    on=merge_cols,
    how="left",
    suffixes=("", "_norm")  # avoids column name clashes
)

# -----------------------------
# 5. Save final dataset
# -----------------------------
final_df.to_excel("Final_merged_dataset.xlsx", index=False)

print("✅ Successfully merged dataset with spatial, temporal, and normalized features!")
print(f"Shape: {final_df.shape}")


✅ Successfully merged dataset with spatial, temporal, and normalized features!
Shape: (77994, 94)


In [2]:
spatial_temporal["datetime_local"] = pd.to_datetime(spatial_temporal["datetime_local"])
normalized["datetime_local"] = pd.to_datetime(normalized["datetime_local"])

spatial_temporal["location_id"] = spatial_temporal["location_id"].astype("int32")
normalized["location_id"] = normalized["location_id"].astype("int32")


In [3]:
final_df.columns[final_df.columns.str.contains("norm")]


Index(['state_norm', 'district_norm', 'location_name_norm',
       'datetime_utc_norm', 'latitude_norm', 'longitude_norm', 'pm25_norm',
       'pm10_norm', 'no2_norm', 'co_norm', 'so2_norm', 'o3_norm',
       'temperature_norm', 'humidity_norm', 'wind_speed_norm',
       'wind_direction_norm', 'Roads_count_norm',
       'Industrial_zones_count_norm', 'Dump_sites_count_norm',
       'Agricultural_fields_count_norm', 'Query_status_norm',
       'Urban_density_score_norm', 'Industrial_presence_norm',
       'Pollution_source_risk_norm', 'Green_area_ratio_norm',
       'dist_nearest_road_m_norm', 'dist_nearest_industry_m_norm',
       'dist_nearest_dump_m_norm', 'dist_nearest_agriculture_m_norm',
       'hour_norm', 'is_peak_hour_norm', 'day_of_week_norm', 'is_weekend_norm',
       'month_norm', 'season_norm', 'season_code_norm', 'pm25_normalized',
       'pm10_normalized', 'no2_normalized', 'co_normalized', 'so2_normalized',
       'o3_normalized', 'temperature_normalized', 'humidity_norm

In [4]:
# Drop all *_norm columns
cols_to_drop = [col for col in final_df.columns if col.endswith("_norm")]
final_df = final_df.drop(columns=cols_to_drop)

print(f"Removed {len(cols_to_drop)} redundant columns")
print(f"New shape: {final_df.shape}")


Removed 36 redundant columns
New shape: (77994, 58)


In [5]:
import pandas as pd

final_df = pd.read_excel("Final_merged_dataset_CLEAN.xlsx")
final_df.head()

,state,district,location_id,location_name,datetime_utc,datetime_local,latitude,longitude,pm25,pm10,...,pm25_scaled,pm10_scaled,no2_scaled,co_scaled,so2_scaled,o3_scaled,temperature_scaled,humidity_scaled,wind_speed_scaled,wind_direction_scaled
0,Andhra Pradesh,Tirupati,5649,"Tirumala, Tirupati - APPCB",2025-11-11T15:00:00Z,2025-11-11 20:30:00,13.67,79.35,81.0,115.0,...,-0.016369,-0.048343,-0.027710,0.093771,-0.372981,-0.005949,0.202941,1.059692,-0.188803,2.060052
1,Andhra Pradesh,Tirupati,5649,"Tirumala, Tirupati - APPCB",2025-11-11T15:15:00Z,2025-11-11 20:45:00,13.67,79.35,81.0,115.0,...,-0.016369,-0.048343,-0.035966,0.001364,-0.372981,-0.005949,0.181327,1.059692,-0.194356,2.060052
2,Andhra Pradesh,Tirupati,5649,"Tirumala, Tirupati - APPCB",2025-11-11T15:45:00Z,2025-11-11 21:15:00,13.67,79.35,81.0,115.0,...,-0.016369,-0.048343,-0.030092,0.186179,-0.369372,-0.005949,0.170521,1.059692,-0.199909,2.060052
3,Andhra Pradesh,Tirupati,5649,"Tirumala, Tirupati - APPCB",2025-11-11T16:15:00Z,2025-11-11 21:45:00,13.67,79.35,90.0,114.0,...,-0.014890,-0.049869,-0.032791,0.067369,-0.372981,-0.005949,0.202941,0.939240,-0.199909,2.060052
4,Andhra Pradesh,Tirupati,5649,"Tirumala, Tirupati - APPCB",2025-11-11T16:30:00Z,2025-11-11 22:00:00,13.67,79.35,90.0,114.0,...,-0.014890,-0.049869,-0.043429,-0.091043,-0.376589,-0.005949,0.192134,0.979391,-0.194356,2.060052


In [7]:
final_df.isnull().sum()

state                             0
district                          0
location_id                       0
location_name                     0
datetime_utc                      0
datetime_local                    0
latitude                          0
longitude                         0
pm25                           5360
pm10                           8972
no2                            6092
co                             9165
so2                            6219
o3                            12926
temperature                   13372
humidity                      14519
wind_speed                    14406
wind_direction                14459
Roads_count                       0
Industrial_zones_count            0
Dump_sites_count                  0
Agricultural_fields_count         0
Query_status                      0
Urban_density_score               0
Industrial_presence               0
Pollution_source_risk             0
Green_area_ratio                  0
dist_nearest_road_m         

In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from shapely.strtree import STRtree
import numpy as np
from tqdm import tqdm

# ---------------------------
# FILE PATHS
# ---------------------------
INPUT_EXCEL = "Final_merged_dataset_CLEAN.xlsx"
OUTPUT_EXCEL = "Finalised_merged_dataset.xlsx"

roads_fp = r"data/roads.osm.pbf"
industrial_fp = r"data/industrial.osm.pbf"
dump_fp = r"data/dump_sites.osm.pbf"
agri_fp = r"data/agriculture.osm.pbf"

# ---------------------------
# 1. LOAD FINAL DATASET
# ---------------------------
print("📥 Loading FINAL dataset...")
df = pd.read_excel(INPUT_EXCEL)

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"
).to_crs(epsg=3857)  # meters

points = gdf.geometry.values
n = len(points)

# ---------------------------
# 2. LOAD OSM FEATURES
# ---------------------------
print("🗺️ Loading OSM layers...")

roads = gpd.read_file(roads_fp, layer="lines").to_crs(3857)

industrial = gpd.read_file(
    industrial_fp, layer="multipolygons"
).to_crs(3857)

dump_sites = gpd.read_file(
    dump_fp, layer="multipolygons"
).to_crs(3857)

agriculture = gpd.read_file(
    agri_fp, layer="multipolygons"
).to_crs(3857)

# ---------------------------
# 3. CLIP TO POINT BOUNDING BOX
# ---------------------------
minx, miny, maxx, maxy = gdf.total_bounds
bbox = box(minx, miny, maxx, maxy)

roads = roads[roads.intersects(bbox)]
industrial = industrial[industrial.intersects(bbox)]
dump_sites = dump_sites[dump_sites.intersects(bbox)]
agriculture = agriculture[agriculture.intersects(bbox)]

print("Features after clip:",
      len(roads), len(industrial), len(dump_sites), len(agriculture))

# ---------------------------
# 4. BUILD STRtree (CORRECT WAY)
# ---------------------------
road_geoms = roads.geometry.values
ind_geoms = industrial.geometry.values
dump_geoms = dump_sites.geometry.values
agri_geoms = agriculture.geometry.values

road_tree = STRtree(road_geoms)
ind_tree = STRtree(ind_geoms)
dump_tree = STRtree(dump_geoms)
agri_tree = STRtree(agri_geoms)

# ---------------------------
# 5. FIXED DISTANCE FUNCTION
# ---------------------------
def compute_min_distance(tree, geometries, pts):
    out = np.empty(len(pts))
    for i, p in enumerate(pts):
        idx = tree.query(p)  # returns INDICES
        if len(idx) == 0:
            out[i] = np.nan
        else:
            out[i] = min(p.distance(geometries[j]) for j in idx)
    return out

# ---------------------------
# 6. BATCH COMPUTATION
# ---------------------------
batch_size = 5000

dist_road = np.empty(n)
dist_ind = np.empty(n)
dist_dump = np.empty(n)
dist_agri = np.empty(n)

print("⏳ Recomputing spatial proximity distances...")
for start in tqdm(range(0, n, batch_size)):
    end = min(start + batch_size, n)
    batch_pts = points[start:end]

    dist_road[start:end] = compute_min_distance(road_tree, road_geoms, batch_pts)
    dist_ind[start:end] = compute_min_distance(ind_tree, ind_geoms, batch_pts)
    dist_dump[start:end] = compute_min_distance(dump_tree, dump_geoms, batch_pts)
    dist_agri[start:end] = compute_min_distance(agri_tree, agri_geoms, batch_pts)

# ---------------------------
# 7. UPDATE ONLY SPATIAL COLUMNS
# ---------------------------
gdf["dist_nearest_road_m"] = dist_road
gdf["dist_nearest_industry_m"] = dist_ind
gdf["dist_nearest_dump_m"] = dist_dump
gdf["dist_nearest_agriculture_m"] = dist_agri

# ---------------------------
# 8. SAVE FIXED DATASET
# ---------------------------
print("💾 Saving corrected FINAL dataset...")
gdf.drop(columns="geometry").to_excel(OUTPUT_EXCEL, index=False)

print("✅ DONE — Spatial proximity columns FIXED in final dataset!")


📥 Loading FINAL dataset...
🗺️ Loading OSM layers...
Features after clip: 9427153 21573 773 84265
⏳ Recomputing spatial proximity distances...


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:10<00:00,  1.48it/s]


💾 Saving corrected FINAL dataset...
✅ DONE — Spatial proximity columns FIXED in final dataset!


In [12]:
print(
    roads.geometry.iloc[0].geom_type,
    industrial.geometry.iloc[0].geom_type,
    dump_sites.geometry.iloc[0].geom_type,
    agriculture.geometry.iloc[0].geom_type
)


LineString MultiPolygon MultiPolygon MultiPolygon


In [13]:
import pandas as pd

df = pd.read_excel("Finalised_merged_dataset.xlsx")

df[
    [
        "dist_nearest_road_m",
        "dist_nearest_industry_m",
        "dist_nearest_dump_m",
        "dist_nearest_agriculture_m"
    ]
].describe()


,dist_nearest_road_m,dist_nearest_industry_m,dist_nearest_dump_m,dist_nearest_agriculture_m
count,70978.000000,13439.000000,0.0,0.0
mean,232.091886,22.968907,NaN,NaN
std,738.065916,37.710140,NaN,NaN
min,0.620448,0.000000,NaN,NaN
25%,13.595581,0.000000,NaN,NaN
50%,26.781288,0.000000,NaN,NaN
75%,65.523674,51.838785,NaN,NaN
max,4008.434126,96.170442,NaN,NaN


In [25]:
import pandas as pd

# Load your final dataset
df = pd.read_excel("Finalised_merged_dataset.xlsx")

# Fill missing distances for dump and agriculture
df["dist_nearest_dump_m"] = df["dist_nearest_dump_m"].fillna(10000)
df["dist_nearest_agriculture_m"] = df["dist_nearest_agriculture_m"].fillna(10000)

# Save changes back to the same file
df.to_excel("Finalised_merged_dataset.xlsx", index=False)

print("✅ Missing spatial proximity values filled and saved in the existing dataset.")


✅ Missing spatial proximity values filled and saved in the existing dataset.


In [26]:
df[
    [
        "dist_nearest_road_m",
        "dist_nearest_industry_m",
        "dist_nearest_dump_m",
        "dist_nearest_agriculture_m"
    ]
].describe()

,dist_nearest_road_m,dist_nearest_industry_m,dist_nearest_dump_m,dist_nearest_agriculture_m
count,70978.000000,13439.000000,77994.0,77994.0
mean,232.091886,22.968907,10000.0,10000.0
std,738.065916,37.710140,0.0,0.0
min,0.620448,0.000000,10000.0,10000.0
25%,13.595581,0.000000,10000.0,10000.0
50%,26.781288,0.000000,10000.0,10000.0
75%,65.523674,51.838785,10000.0,10000.0
max,4008.434126,96.170442,10000.0,10000.0


In [22]:
print("Data extraction and merging into final dataset completed!")

Data extraction and merging into final dataset completed!
